In [ ]:
%matplotlib inline

# Import dependencies

We begin by importing the necessary libraries.

In [ ]:
# System & OS
import os
import glob
# Data Storage
from google.colab import drive
from zipfile import ZipFile, is_zipfile
# Data Analysis
import numpy as np
import pandas as pd
# Data Visualization
from tqdm import tqdm
import matplotlib.pyplot as plt

# Mount Storage

For simplicity, we keep the data in a Google Drive folder, and simply mount the Google Drive to our Colab instance, as if it were a local file system.

In [ ]:
# Mount Google Drive to Colab Instance
drive.mount('/content/drive')
# Change directory to where the data are stored
%cd '/content/drive/MyDrive/Research/Ongoing/Protostellar Luminosity/Data'

Mounted at /content/drive
/content/drive/MyDrive/Research/Ongoing/Protostellar Luminosity/Data


Unzip the files, if we haven't done so already.

In [ ]:
if not os.path.exists('models'):
  file_to_extract = 'models.zip'

  if os.path.exists(file_to_extract):
      if is_zipfile(file_to_extract):
          print(f'Valid zip file.\nExtracting: {file_to_extract}')
          # Read in zip file
          with ZipFile(file_to_extract,'r') as zip_ref:
              # Add progress bar
              for file in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
                  # Extract and store in current directory
                  zip_ref.extract(member=file)
      else:
          print('Not valid zip file.')
  else:
      print(f'Cannot find: {file_to_extract}.')
else:
  print('Data ready.')

Data ready


# Data Analysis

Define lists to store every model number, and every inclination value, as well as an empty list to store (model number, class) tuples, for later use.

In [ ]:
model_num_list = ["01", "02", "10", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "30", "31", "32", "33"]
inclinations = ["05", "15", "25", "35", "45", "55", "65", "75", "85"]

# Declare a 2D list to store tuples of (model, class) split indeces
split_tuples = [[0 for x in range(2)] for y in range(len(model_num_list))]

We create a master file to store pertinent model data, and start by writing the header row with relevant column labels. 

In [ ]:
with open("master_file.tbl", 'w') as master_file:
  master_file.write("{:<15} {:<20} {:<15} {:<20} {:<25} {:<15}".format("L_int", 
                                                                       "Flux", 
                                                                       "Inclination", 
                                                                       "Wavelength of flux", 
                                                                       "Internal vs Final mass", 
                                                                       "Model Number"))
  master_file.write('\n')
master_file.close()

Next, loop over every model, reading in the appropriate data, and doing the necessary computations. 

The end result is a fully populated master file, along with some useful values stored to memory. 

In [ ]:
# Loop over every model
for i in range(len(model_num_list)):
  # The number of timesteps is different for each model; num_times is the result of taking the number of .dat files in each model's RESULTS directory, and dividing by the number of inclinations (to get the number of timesteps)
  num_times = int(len(glob.glob1(f'models/run_evolpapiii_newdisk_model{model_num_list[i]}/RESULTS/','*.dat')) / len(inclinations))
  # Then, loop from 2 (every model's initial timestep is 2) to num_times + 2 (so it stops at num_times + 1) to get a list with every timestep
  timesteps = []
  for a in range (2, num_times + 2):
    timesteps.append(str(a)) 

  # Read in the internal luminosity data from the luminosities file
  df = pd.read_table(f'models/luminosities_model{model_num_list[i]}.tbl', 
                     skiprows=1, 
                     delim_whitespace=True, 
                     names=['Time (Myr)',
                            'Time-t0 (yr)',
                            'L_EtoD (Lsun)',
                            'L_DM (Lsun)',
                            'L_DtoS (Lsun)',
                            'L_EtoS (Lsun)',
                            'L_DR (Lsun)',
                            'L_PHOT (Lsun)',
                            'L_INT (Lsun)'])
  lint = df['L_INT (Lsun)']
  # Read in the mass of the star, disk, and envelope data from the parameters file
  df = pd.read_table(f'models/model_parameters_model{model_num_list[i]}.tbl', 
                     skiprows=1, 
                     delim_whitespace=True, 
                     names=['Time (Myr)',
                            'Time-t0 (yr)',
                            'Mstar (Msun)',
                            'Lstar (Lsun)',
                            'Rstar (Rsun)',
                            'Tstar (K)',
                            'Rdisk_in (AU)',
                            'Rdisk_out (AU)',
                            'Mdisk (Msun)',
                            'Renv_in (AU)',
                            'Renv_out (AU)',
                            'Menv (Msun)',
                            'Omega_0 (1/s)',
                            'c_s (cm/s)'])
  mStar, mDisk, mEnv = df['Mstar (Msun)'], df['Mdisk (Msun)'], df['Menv (Msun)']
  # Initialize variables to store class split index, and to keep track of the number of "missing" timesteps across all models
  split, missing_timesteps = 0, 0

  # Open master file to write in pertinent data
  with open("master_file.tbl", 'a') as master_file:
    # Loop over every spectrum file
    for b in range(len(timesteps)):  
      for c in range(len(inclinations)):
        # Read in the frequency and flux data from the spectrum file
        if os.path.isfile(f'models/run_evolpapiii_newdisk_model{model_num_list[i]}/RESULTS/spectrum_{timesteps[b]}_inc{inclinations[c]}.dat') == True: 
          df = pd.read_table(f'models/run_evolpapiii_newdisk_model{model_num_list[i]}/RESULTS/spectrum_{timesteps[b]}_inc{inclinations[c]}.dat', 
                             skiprows=2, 
                             delim_whitespace=True, 
                             names=['Frequency',
                                    'Flux'])
          frequency, flux = df['Frequency'], df['Flux']
          # Adjust flux data to be consistent with Dunham's previous work
          for d in range(len(flux)):
            flux[d] = flux[d] * frequency[d] * (1.0 / 140.0)**2
          # Loop over every frequency value, to find the wavelength closest to 70 microns
          min = 0
          for e in range(len(frequency)):
            wavelength_test = 2.99792458e14 / frequency[e]
            min_test = abs(70 - wavelength_test)
            if(min == 0) or (min_test < min):
              min = min_test
              wavelength = wavelength_test
              index = e
          # Writing data
          master_file.write("{:<15} {:<20} {:<15} {:<20} {:<25} {:<15}".format(lint[int(timesteps[b]) - 1], flux[index], inclinations[c], wavelength, (mStar[b] + mDisk[b]) / (mStar[b] + mDisk[b] + mEnv[b]), model_num_list[i]))
          master_file.write('\n')
          # Find the point (index) where the star goes from class 0 to 1
          if split == 0:
            if (mStar[b] + mDisk[b]) / (mStar[b] + mDisk[b] + mEnv[b]) >= 0.5:
              split_tuples[i][1] = b * len(inclinations)
        else:
          missing_timesteps += 1
  master_file.close()

  # Find the point (index) where the current model ends
  if i == 0:
      split_tuples[i][0] = len(timesteps) * len(inclinations) - missing_timesteps
  else:
      split_tuples[i][0] = split_tuples[i - 1][0] + len(timesteps) * len(inclinations) - missing_timesteps

# Data Visualization

To start the visualization process, we first read back in the data from our master file.

In [ ]:
df = pd.read_table('master_file.tbl', skiprows=1, delim_whitespace=True, names=['L_int', 
                                                                           'Flux', 
                                                                           'Inclination', 
                                                                           'Wavelength of flux', 
                                                                           'Internal vs Final mass', 
                                                                           'Model Number'])
lint_master, flux_master = df['L_int'], df['Flux']
# Initialize master lists to store all class 0 and 1 data
lint_master_0, flux_master_0, lint_master_1, flux_master_1, lint_master_log, flux_master_log, lint_master_0_log, flux_master_0_log, lint_master_1_log, flux_master_1_log = [], [], [], [], [], [], [], [], [], []

Then, loop over each model's data, making 3 plots each (Class 0, Class 1, and Class 0 & 1), all fitted with a linear regression.

In [ ]:
# Loop over each tuple in the 2D list split_tuples, such that we consdier only one model at a time
for j in range(len(model_num_list)):
  # Split out appropriate internal luminosity and flux data
  if j == 0:
    lint, flux = lint_master[:split_tuples[j][0]], flux_master[:split_tuples[j][0]]
  else:
    lint, flux = lint_master[split_tuples[j - 1][0]:split_tuples[j][0]], flux_master[split_tuples[j - 1][0]:split_tuples[j][0]]
  # Split data into class 0 and 1
  lint_0, flux_0, lint_1, flux_1 = lint[:split_tuples[j][1]], flux[:split_tuples[j][1]], lint[split_tuples[j][1]:], flux[split_tuples[j][1]:]
  # Take the log (base 10) of each flux and internal luminosity list
  lint_log, flux_log, lint_0_log, flux_0_log, lint_1_log, flux_1_log = np.log10(lint), np.log10(flux), np.log10(lint_0), np.log10(flux_0), np.log10(lint_1), np.log10(flux_1)
  # Add data to master lists
  lint_master_0.extend(lint_0)
  flux_master_0.extend(flux_0)
  lint_master_1.extend(lint_1)
  flux_master_1.extend(flux_1)
  lint_master_log.extend(lint_log)
  flux_master_log.extend(flux_log)
  lint_master_0_log.extend(lint_0_log)
  flux_master_0_log.extend(flux_0_log)
  lint_master_1_log.extend(lint_1_log)
  flux_master_1_log.extend(flux_1_log)

  # Make flux vs internal luminosity plots, starting first with Class 0 & 1
  plt.scatter(lint, flux)
  plt.title(f'Model {model_num_list[j]} Flux vs Internal Luminosity (Class 0 & 1)')
  plt.xlabel('Internal Luminosity (L$_{sun}$)')
  plt.ylabel('Flux (erg cm$^{-2}$ s$^{-1}$)')
  plt.xscale('log')
  plt.yscale('log')
  plt.xlim([10**(-8), 10**2])
  plt.ylim([10**(-22), 10**(-14)])

  # Plot linear regression, where we only consider internal luminosity values >= 0.1
  split_lin_reg = 0
  for f in range(len(lint)):
    if split_lin_reg == 0:
      if lint[f] >= 0.1:   
        split_lin_reg = f
  # Split the logged data such that internal luminosity values < 0.1 are rejected
  lint_log_lin_reg, flux_log_lin_reg = lint_log[split_lin_reg:], flux_log[split_lin_reg:]
  # Fit the linear regression to the logged data, yielding a slope and intercept (keeping in mind to plot against linear data)
  m, b = np.polyfit(lint_log_lin_reg, flux_log_lin_reg, 1)
  L = np.logspace(10 ** -8, 10 ** 2, 846)
  F = (10 ** b) * (L ** m)
  plt.plot(L, F, color='red')

  # Save the plot
  plt.savefig(f'Figures/model_{model_num_list[j]}_flux_vs_lint_master.eps')
  
  # Next, for Class 0
  plt.scatter(lint_0, flux_0)
  plt.title(f'Model {model_num_list[j]} Flux vs Internal Luminosity (Class 0)')
  plt.xlabel('Internal Luminosity (L$_{sun}$)')
  plt.ylabel('Flux (erg cm$^{-2}$ s$^{-1}$)')
  plt.xscale('log')
  plt.yscale('log')
  plt.xlim([10**(-8), 10**2])
  plt.ylim([10**(-22), 10**(-14)])

  # Plot linear regression, where we only consider internal luminosity values >= 0.1
  split_lin_reg = 0
  for g in range(len(lint_0)):
    if split_lin_reg == 0:
      if lint_0[g] >= 0.1:
        split_lin_reg = g
  # Split the logged data such that internal luminosity values < 0.1 are rejected
  lint_0_log_lin_reg, flux_0_log_lin_reg = lint_0_log[split_lin_reg:], flux_0_log[split_lin_reg:]
  # Fit the linear regression to the logged data, yielding a slope and intercept (keeping in mind to plot against linear data)
  m, b = np.polyfit(lint_0_log_lin_reg, flux_0_log_lin_reg, 1)
  L = np.logspace(10 ** -8, 10 ** 2, 846)
  F = (10 ** b) * (L ** m)
  plt.plot(L, F, color='red')

  # Save the plot
  plt.savefig(f'Figures/model_{model_num_list[j]}_flux_vs_lint_class0.eps')

  # Finally, for class 1
  plt.scatter(lint_1, flux_1)
  plt.title(f'Model {model_num_list[j]} Flux vs Internal Luminosity (Class 1)')
  plt.xlabel('Internal Luminosity (L$_{sun}$)')
  plt.ylabel('Flux (erg cm$^{-2}$ s$^{-1}$)')
  plt.xscale('log')
  plt.yscale('log')
  plt.xlim([10**(-8), 10**2])
  plt.ylim([10**(-22), 10**(-14)])

  # Plot linear regression, where we note that all internal luminosity values are >= 0.1, meaning we can simply fit the linear regression to the logged data, yielding a slope and intercept (keeping in mind to plot against linear data)
  m, b = np.polyfit(lint_1_log, flux_1_log, 1)
  L = np.logspace(10 ** -8, 10 ** 2, 846)
  F = (10 ** b) * (L ** m)
  plt.plot(L, F, color='red')

  # Save the plot
  plt.savefig(f'Figures/model_{model_num_list[j]}_flux_vs_lint_class1.eps')

Finally, make 3 master plots containing all the data from every single model (Class 0, Class 1, and Class 0 & 1), again fitting a linear regression to each.

In [ ]:
# Class 0 & 1
plt.scatter(lint_master, flux_master)
plt.title('All Models Flux vs Internal Luminosity (Class 0 & 1)')
plt.xlabel('Internal Luminosity (L$_{sun}$)')
plt.ylabel('Flux (erg cm$^{-2}$ s$^{-1}$)')
plt.xscale('log')
plt.yscale('log')
plt.xlim([10**(-8), 10**2])
plt.ylim([10**(-22), 10**(-14)])

# Plot linear regression, where we only consider internal luminosity values >= 0.1
split_lin_reg = 0
for h in range(len(lint_master)):
  if split_lin_reg == 0:
    if lint_master[h] >= 0.1:   
      split_lin_reg = h
# Split the logged data such that internal luminosity values < 0.1 are rejected
lint_master_log_lin_reg, flux_master_log_lin_reg = lint_master_log[split_lin_reg:], flux_master_log[split_lin_reg:]
# Fit the linear regression to the logged data, yielding a slope and intercept (keeping in mind to plot against linear data)
m, b = np.polyfit(lint_master_log_lin_reg, flux_master_log_lin_reg, 1)
L = np.logspace(10 ** -8, 10 ** 2, 846)
F = (10 ** b) * (L ** m)
plt.plot(L, F, color='red')

# Save the plot
plt.savefig('Figures/flux_vs_lint_master.eps')

# Class 0 
plt.scatter(lint_master_0, flux_master_0)
plt.title('All Models Flux vs Internal Luminosity (Class 0)')
plt.xlabel('Internal Luminosity (L$_{sun}$)')
plt.ylabel('Flux (erg cm$^{-2}$ s$^{-1}$)')
plt.xscale('log')
plt.yscale('log')
plt.xlim([10**(-8), 10**2])
plt.ylim([10**(-22), 10**(-14)])

# Plot linear regression, where we only consider internal luminosity values >= 0.1
split_lin_reg = 0
for i in range(len(lint_master_0)):
  if split_lin_reg == 0:
    if lint_master_0[i] >= 0.1:
      split_lin_reg = g
# Split the logged data such that internal luminosity values < 0.1 are rejected
lint_master_0_log_lin_reg, flux_master_0_log_lin_reg = lint_master_0_log[split_lin_reg:], flux_master_0_log[split_lin_reg:]
# Fit the linear regression to the logged data, yielding a slope and intercept (keeping in mind to plot against linear data)
m, b = np.polyfit(lint_master_0_log_lin_reg, flux_master_0_log_lin_reg, 1)
L = np.logspace(10 ** -8, 10 ** 2, 846)
F = (10 ** b) * (L ** m)
plt.plot(L, F, color='red')

# Save the plot
plt.savefig('Figures/flux_vs_lint_master_class0.eps')

# Class 1
plt.scatter(lint_master_1, flux_master_1)
plt.title('All Models Flux vs Internal Luminosity (Class 1)')
plt.xlabel('Internal Luminosity (L$_{sun}$)')
plt.ylabel('Flux (erg cm$^{-2}$ s$^{-1}$)')
plt.xscale('log')
plt.yscale('log')
plt.xlim([10**(-8), 10**2])
plt.ylim([10**(-22), 10**(-14)])

# Plot linear regression, where we note that all internal luminosity values are >= 0.1, meaning we can simply fit the linear regression to the logged data, yielding a slope and intercept (keeping in mind to plot against linear data)
m, b = np.polyfit(lint_master_1_log, flux_master_1_log, 1)
L = np.logspace(10 ** -8, 10 ** 2, 846)
F = (10 ** b) * (L ** m)
plt.plot(L, F, color='red')

# Save the plot
plt.savefig('Figures/flux_vs_lint_master_class1.eps')